In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/merck-datasets/Datasets/acs_housing_5yr.csv
/kaggle/input/merck-datasets/Datasets/treatments_2017-2020.csv
/kaggle/input/merck-datasets/Datasets/treatment_facilities_2016_2020.csv
/kaggle/input/merck-datasets/Datasets/state_county_cbsa_population.csv
/kaggle/input/merck-datasets/Datasets/acs_demographics_1yr.csv
/kaggle/input/merck-datasets/Datasets/acs_social_5yr.csv
/kaggle/input/merck-datasets/Datasets/acs_economics_1yr.csv
/kaggle/input/merck-datasets/Datasets/acs_demographics_5yr.csv
/kaggle/input/merck-datasets/Datasets/acs_social_1yr.csv
/kaggle/input/merck-datasets/Datasets/acs_economics_5yr.csv
/kaggle/input/merck-datasets/Datasets/acs_housing_1yr.csv


In [2]:
path = '/kaggle/input/merck-datasets/Datasets/'

In [3]:
treatments = pd.read_csv(path +'treatments_2017-2020.csv',low_memory=False)

cleaning: (1) check null? (2) remove -9 appearances in columns studied

In [4]:
treatments.isnull().values.any()

False

Preprocessing: prepare a table of dropout studies: <br>
(1) keep columns interested <br>
(2) add total num of substances as a new variable (3) LOS - map value,
(4) REASON map to DROPOUT as Yes or No, remove rows if REASON unknown
<br>
(5) survival curve -- keep 365 days or less (6) for regression or classification - convert categorical variables

In [5]:
cols = ['REASON', 
        'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'DSMCRIT','FREQ_ATND_SELF_HELP',
        'DIVISION', 'CBSA',
        'NOPRIOR', 'AGE', 'GENDER', 'MARSTAT', 'RACE', 'EDUC', 'PREG', 'VET', 
        'LIVARAG', 'ARRESTS', 'PSYPROB',
        'HLTHINS', 'EMPLOY', 'PRIMPAY',
        'LOS', 'SERVICES', 'IDU', 'DAYWAIT', 'PSOURCE',
        'ALCFLG','COKEFLG','MARFLG','HERFLG','METHFLG','OPSYNFLG','PCPFLG',
        'HALLFLG','MTHAMFLG','AMPHFLG','STIMFLG','BENZFLG','TRNQFLG','BARBFLG',
        'SEDHPFLG','INHFLG','OTCFLG','OTHERFLG',
         'METHUSE'
       ]

los_convert = {
    31: 38,
    32: 53,
    33: 75.5,
    34: 105.5,
    35: 150.5,
    36: 273
}

In [6]:
def df_prepare(treatments):
    # note: df name must be changed if rows deleted!
    
    treatments1 = treatments[cols]
    
    ## add num of total drugs col
    FLGs = treatments[['ALCFLG','COKEFLG','MARFLG','HERFLG','METHFLG','OPSYNFLG','PCPFLG','HALLFLG','MTHAMFLG','AMPHFLG','STIMFLG','BENZFLG','TRNQFLG','BARBFLG','SEDHPFLG','INHFLG','OTCFLG','OTHERFLG']];
    total = FLGs.sum(axis=1, numeric_only=True)
    treatments1['totalSUB'] = total
    
    ## map LOS to actual days
    treatments1['LOS1'] = treatments1['LOS'].replace(los_convert)
    ## remove REASON unknown rows
    treatments2 = treatments1[treatments1['REASON'] != 7]
    ## Attrition col. as dropout YES/NO
    treatments2['Attrition'] = np.where(treatments2['REASON'] == 2, "Yes", "No")
    
    return treatments2

def explore(treatments2):
    
    treatments_dropout = treatments2[treatments2['REASON'] == 2]
    #treatments_dropout['LOS1'].describe()
    treatments_dropout['LOS1'].hist(grid=True, bins=np.arange(0, 370, 10))
    plt.title('Length of stay for dropout cases')

### classification - DROPOUT Yes/No

In [7]:
def prepare_balanced_df(treatments2):
    #keep a number of rows of Non dropout as the same size of dropout rows
    
    count_dropout = treatments2[treatments2['Attrition'] == "Yes"].shape[0]
    non_drop = treatments2[treatments2['Attrition'] == "No"]
    cases_drop = treatments2[treatments2['Attrition'] == "Yes"]
    cases_sample_non_drop = non_drop.sample(count_dropout)
    tr_sample = pd.concat([cases_drop, cases_sample_non_drop], axis=0)
    
    return tr_sample

In [8]:
# x_cols = [ 'totalSUB',
# #           'CBSA',
#           'SUB1', 'ROUTE1', 'FREQ1', 'DSMCRIT','FREQ_ATND_SELF_HELP',
#          'DIVISION', 'NOPRIOR', 'GENDER',            
#         'LIVARAG',  'PSYPROB',
#         'HLTHINS', 'EMPLOY', 'PRIMPAY',
#         'LOS1', 'SERVICES', 'IDU',  'PSOURCE', 
#           ## low corr.
# #           'FRSTUSE1', 'MARSTAT','RACE', 'EDUC','PREG', 'VET','ARRESTS',
# #           'DAYWAIT', 'AGE',
#          ]

x_cols = [
    'ROUTE1', 'FREQ1', 'DSMCRIT','FREQ_ATND_SELF_HELP',
    'DIVISION', 'NOPRIOR', 'GENDER',            
     'LIVARAG',  'PSYPROB',
    'HLTHINS', 'EMPLOY', 'PRIMPAY',
     'LOS1', 'SERVICES', 'IDU',  'PSOURCE',
    'TRNQFLG', 'INHFLG', 'HERFLG', 'SEDHPFLG', 'BENZFLG', 'OPSYNFLG', 
    'ALCFLG', 'OTCFLG', 'AMPHFLG', 'PCPFLG', 'COKEFLG', 'MTHAMFLG', 
    'MARFLG', 'STIMFLG', 'BARBFLG', 'HALLFLG', 'METHFLG', 'OTHERFLG',
    'DAYWAIT', 'METHUSE'
    ]

## categorical - convert to dummies
cols_convert = [ #'CBSA', 'SUB1',
                 'ROUTE1', 'FREQ1', 'DSMCRIT','FREQ_ATND_SELF_HELP', 
            'DIVISION', 'NOPRIOR', 'GENDER',
             'LIVARAG',  'PSYPROB',
             'HLTHINS', 'EMPLOY', 'PRIMPAY',
             'SERVICES', 'IDU',  'PSOURCE',
            'DAYWAIT', 'METHUSE'            
            ]

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

def prep_df_classify(treatments2):
    
    ## use all cases
    tr_clf = treatments2[x_cols +['Attrition']]
    ## use sampled tr
    #tr_clf = tr_sample[x_cols +['Attrition']]
    
    encoder = LabelEncoder()
    tr_clf['Attrition1'] = encoder.fit_transform(tr_clf['Attrition'])
    
    tr_clf_num = pd.get_dummies(tr_clf, columns=cols_convert) 
## this is the df to use in clf/reg

    ## remove LOS over 365 days cases
    tr_clf_num = tr_clf_num[tr_clf_num['LOS1'] != 37]

    y_col_clf = ['Attrition1']
    x_cols_clf = [ x for x in tr_clf_num.columns.tolist() if "Attrition" not in x]

    ## remove "-9" cols (unknown)
    x_cols_clf = [ x for x in x_cols_clf if "_-9" not in x]

    X = tr_clf_num[x_cols_clf]
    y = tr_clf_num[y_col_clf]
    y = y.values.ravel()  # y has to be array

    return X, y

def classify(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

    clf = lgb.LGBMClassifier()
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)

    accuracy=accuracy_score(y_pred, y_test)
    print('accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

    n_features = len(X_train.columns)
    feature_importance_pairs = []
    for i in range(n_features):
        feature_importance_pairs.append((clf.feature_importances_[i], X_train.columns[i]))

    feature_importance_pairs.sort(reverse=True)
    print(*feature_importance_pairs, sep="\n")
    
    print(cross_val_score(clf, X, y, cv=5))

In [10]:
## kfold cv
def kfoldCV(X, y):
    
    N_SPLITS = 5
    strat_kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=208)

    #scores = np.empty(N_SPLITS)

    for idx, (train_idx, test_idx) in enumerate(strat_kf.split(X, y)):
        print("=" * 12 + f"Training fold {idx}" + 12 * "=")

        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y[train_idx], y[test_idx]
        eval_set = [(X_val, y_val)]

        lgbm_clf = lgb.LGBMClassifier(n_estimators=100)
        lgbm_clf.fit(
            X_train,
            y_train,
            eval_set=eval_set,
            #eval_metric="binary_logloss",
        )

        preds = lgbm_clf.predict(X_val)  # predict_proba() if use log_loss
        #loss = log_loss(y_val, preds)
        print("accu. ", accuracy_score(y_val, preds))
        #scores[idx] = loss

        #print(f"Fold {idx} finished with logloss score: {loss:.5f} \n")
    

### regression for LOS (act. days not category) 

In [12]:
from sklearn.metrics import r2_score

def regression(treatments2):
    ## preprocess: need to take only dropout cases; remove LOS over 365 days
    
    tr_clf = treatments2[x_cols +['Attrition']]
    ## use sampled tr
    #tr_clf = tr_sample[x_cols +['Attrition']]
    
    encoder = LabelEncoder()
    tr_clf['Attrition1'] = encoder.fit_transform(tr_clf['Attrition'])
    
    tr_clf_num = pd.get_dummies(tr_clf, columns=cols_convert) 

    ## remove LOS over 365 days cases
    tr_clf_num = tr_clf_num[tr_clf_num['LOS1'] != 37]
    tr_reg_num = tr_clf_num[tr_clf_num['Attrition'] == 'Yes']
    
    x_cols_clf = [ x for x in tr_clf_num.columns.tolist() if "Attrition" not in x]
    ## remove "-9" cols (unknown)
    x_cols_clf = [ x for x in x_cols_clf if "_-9" not in x]
    x_cols_reg = [ x for x in x_cols_clf if "LOS" not in x]
    y_col_reg = ['LOS1']
    
    X = tr_reg_num[x_cols_reg]
    y = tr_reg_num[y_col_reg]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

    reg = lgb.LGBMRegressor()
    reg.fit(X_train, y_train)
    y_pred=reg.predict(X_test)

    print(r2_score(y_test, y_pred))

## Survival analysis, T as drop out time

In [14]:
!pip install lifelines
import lifelines # for survival analysis
from lifelines import KaplanMeierFitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.4/409.4 kB 7.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=c1279cf98d22442d7a9737538e4473eb2d114d0f371a30f02e9456db74e174cd
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [15]:
def survivalKMF(treatments_comp):
    
    # only study <= 365 days cases
    treatments_comp = treatments_comp[treatments_comp['LOS'] < 37]
    encoder = LabelEncoder()
    treatments_comp['Attrition1'] = encoder.fit_transform(treatments_comp['Attrition'])
    
    kmf = KaplanMeierFitter()
    kmf.fit(durations=treatments_comp['LOS1'], event_observed=treatments_comp['Attrition1'])

    kmf.survival_function_.plot(figsize=(8,5))
    plt.title('Survival Curve estimated with Kaplan-Meier Fitter')
    plt.xlabel('Length of stay')
    plt.show()

    kmf.plot_survival_function(figsize=(8,5))
    plt.title('Survival Curve estimated with Kaplan-Meier Fitter with confidence intervals')
    plt.xlabel('Length of stay')
    plt.show()

    kmf.survival_function_

In [ ]:
## main()

df2 = df_prepare(treatments)
# X, y = prep_df_classify(df2)
# classify(X, y)
# kfoldCV(X, y)
# regression(df2)
survivalKMF(df2)

accuracy score: 0.7988 -- all cases <br>
0.7594 -- balanced drop/nondrop